In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os

In [5]:
# Load MoveNet model
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [75]:
# Define arm keypoints indices
LEFT_ARM_INDICES = [5, 7, 9, 11, 13, 15]
RIGHT_ARM_INDICES = [6, 8, 10, 12, 14, 16]

# Define threshold distance for contact detection
CONTACT_THRESHOLD = 60

# Define rendering functions
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    num_people = len(keypoints_with_scores)
    for i in range(num_people):
        person1 = keypoints_with_scores[i]
        draw_connections(frame, person1, edges, confidence_threshold)
        draw_keypoints(frame, person1, confidence_threshold)
        for j in range(i+1, num_people):
            person2 = keypoints_with_scores[j]
            if detect_contact(person1, person2):
                highlight_contact(frame, person1, person2, True)
            else:
                highlight_contact(frame, person1, person2, False)

def detect_contact(keypoints1, keypoints2):
    # Check if the hands keypoints of person1 are visible and have high enough confidence score
    has_left_hand1 = all(keypoints1[i, 2] > 0.2 for i in LEFT_ARM_INDICES[:2])
    has_right_hand1 = all(keypoints1[i, 2] > 0.2 for i in RIGHT_ARM_INDICES[:2])
    if not (has_left_hand1 or has_right_hand1):
        return False
    
    # Check if any keypoints of person2 are visible and have high enough confidence score
    has_visible_keypoints2 = any(keypoints2[i, 2] > 0.2 for i in range(keypoints2.shape[0]))
    if not has_visible_keypoints2:
        return False
    
    # Check if any hand of person1 is in contact with any body part of person2
    if has_left_hand1:
        left_hand1 = keypoints1[9]
        for i in range(keypoints2.shape[0]):
            body_part2 = keypoints2[i]
            distance = np.sqrt(np.sum(np.square(left_hand1[:2] - body_part2[:2])))
            if distance < CONTACT_THRESHOLD:
                return True
    if has_right_hand1:
        right_hand1 = keypoints1[10]
        for i in range(keypoints2.shape[0]):
            body_part2 = keypoints2[i]
            distance = np.sqrt(np.sum(np.square(right_hand1[:2] - body_part2[:2])))
            if distance < CONTACT_THRESHOLD:
                return True
    
    return False

def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
          
        ky, kx, kp_conf = kp
        
        if kp in shaped[LEFT_ARM_INDICES] or kp in shaped[RIGHT_ARM_INDICES]:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)
            
        elif kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

def highlight_contact(frame, keypoints1, keypoints2, is_contact=False):
    y, x, c = frame.shape
    shaped1 = np.squeeze(np.multiply(keypoints1, [y,x,1]))
    shaped2 = np.squeeze(np.multiply(keypoints2, [y,x,1]))
    
    # Highlight left hand of person 1
    if all(shaped1[i, 2] > 0.2 for i in LEFT_ARM_INDICES[:2]):
        left_hand1 = shaped1[9]
        if has_contact_with_body(left_hand1, shaped2) and is_contact:
            cv2.circle(frame, (int(left_hand1[1]), int(left_hand1[0])), 20, (120,0,120), -1)
        else:
            cv2.circle(frame, (int(left_hand1[1]), int(left_hand1[0])), 20, (0,0,255), -1)
        
    # Highlight right hand of person 1
    if all(shaped1[i, 2] > 0.2 for i in RIGHT_ARM_INDICES[:2]):
        right_hand1 = shaped1[10]
        if has_contact_with_body(right_hand1, shaped2) and is_contact:
            cv2.circle(frame, (int(right_hand1[1]), int(right_hand1[0])), 20, (120,0,120), -1)
        else:
            cv2.circle(frame, (int(right_hand1[1]), int(right_hand1[0])), 20, (0,0,255), -1)

def has_contact_with_body(hand, keypoints):
    for i in range(keypoints.shape[0]):
        if i not in LEFT_ARM_INDICES and i not in RIGHT_ARM_INDICES:
            body_part = keypoints[i]
            distance = np.sqrt(np.sum(np.square(hand[:2] - body_part[:2])))
            if distance < CONTACT_THRESHOLD:
                return True
    return False

In [77]:
# Open video file
cap = cv2.VideoCapture('ufc.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    
    #slow down the video
    # os.system('sleep 1')
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 128,256)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints and detect punches
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.35)
    
    # Display image
    cv2.imshow('Punch Detection', frame)
    
    # Check for quit key
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()

In [21]:
# Define edges for rendering
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}